In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 17.8MB/s 
     |████████████████████████████████| 3.2MB 55.0MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1df986753f1641bfd327836329af5c3002d816c3eee8c70a393e8d0963a2160a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import torch
from torch.cuda import amp
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.optim as optim
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.utils.class_weight import compute_class_weight

from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm


In [ ]:
df = pd.read_excel('drive/MyDrive/euthority/annotation_data_brkdwn.xlsx', index_col=0)

In [ ]:
print(df.columns)
annotations_map = {'group_1' : {
  "e_8" : "Party-refuse-request",
  "e_3" : "Court-consider",
  "e_2" : "Party-request",
  "e_7" : "No_EU_law",
  "e_5" : "Court-refuse",
  "e_6" : "Court-submit",
  "e_4" : "Court-apply",
  "e_1" : "Party-invoke",
  0 : "No_EU_law"
}, 
'group_2': {
  "e_8" : "Party-request",
  "e_12" : "Court-submit",
  "e_7" : "Party-invoke",
  "e_14" : "Party-refuse-request",
  "e_11" : "Court-refuse",
  "e_9" : "Court-consider",
  "e_10" : "Court-apply",
  "e_13" : "No_EU_law",
  0 : "No_EU_law"
}
}

Index(['article', 'class', 'sum_text_len', 'paragraph', 'group'], dtype='object')


In [ ]:
df.dropna(inplace=True)
df['class'] = df['class'].apply(lambda x : ast.literal_eval(x))
print(df[['class','paragraph']])
# classes = df['class'].unique()
# print(classes)

        class                                          paragraph
0       [e_7]  (i) de definitie van de kostencategorieën die ...
1       [e_4]  - de richtlijn 2009/28/EG van het Europees Par...
2       [e_7]  - de vaststelling van de tariefmethodologieën ...
3       [e_4]  - de verordening (EG) nr. 713/2009 van het Eur...
4       [e_7]  - groei van de onafhankelijkheid van de CREG;-...
...       ...                                                ...
18188  [e_13]  Bij arrest van 7 januari 2016 oordeelt het Hof...
18189  [e_13]  De beslissing waarbij de rechtbank de gerechte...
18190  [e_13]  In afwijking van de artikelen 5 en 6, 1°, van ...
18191  [e_13]  Tot slot is de maatregel evenredig in het lich...
18192  [e_13]  § 2. De partij of de notaris-vereffenaar die m...

[18192 rows x 2 columns]


In [ ]:
new_df = pd.DataFrame()
for i in df.group.unique():
    sub_df = df[df['group'] == i]
    sub_df.loc[:, 'category_name'] = sub_df['class'].apply(lambda x : [annotations_map[i][el] for el in x]  )
    # print(sub_df[[ 'category_name', 'paragraph']].head())
    new_df = new_df.append(sub_df)

new_df.reset_index(inplace=True, drop=True)
# print(new_df[['category_name', 'paragraph']].head(10))
del sub_df

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
tdf = new_df.copy()
print(tdf.columns)
del new_df

Index(['article', 'class', 'sum_text_len', 'paragraph', 'group',
       'category_name'],
      dtype='object')


In [ ]:
tdf.category_name.value_counts()

TypeError: ignored

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[No_EU_law]                                                                                       12201
[Party-invoke]                                                                                     2758
[Court-apply]                                                                                      2225
[Court-submit]                                                                                      229
[Party-request]                                                                                     215
[Court-consider]                                                                                    210
[Party-refuse-request]                                                                               59
[Court-refuse]                                                                                       39
[Party-invoke, Court-apply]                                                                          37
[Party-request, Party-invoke]                                   

In [ ]:
tdf['target'] = tdf['category_name'].apply(lambda x : 0 if x == ['No_EU_law'] else 1)
print(tdf['target'].value_counts())
print(tdf.columns)

0    12201
1     5991
Name: target, dtype: int64
Index(['article', 'class', 'sum_text_len', 'paragraph', 'group',
       'category_name', 'target'],
      dtype='object')


In [ ]:
print(tdf[['article']].head(25))

NameError: ignored

In [ ]:
tdf = tdf[['article', 'paragraph', 'target']]
print(tdf.columns)

Index(['article', 'paragraph', 'target'], dtype='object')


In [ ]:
# sdf = tdf[tdf.target == 1]
# # art1 = list(sdf.article.unique())
# print(sdf.shape)
# # sdf2 = tdf[~tdf.article.isin(art1)]
# sdf2 = tdf[tdf.target == 0]
# # art2 = list(sdf2.article.unique())
# print(sdf2.shape)
# # print(len(art1), len(art2)) 

In [ ]:
# trdf = sdf.sample(frac=0.8).reset_index()
# trdf = trdf.append(sdf2.sample(frac=0.8)).reset_index()
# print(trdf.target.value_counts())

In [ ]:
# tedf = tdf[~tdf['paragraph'].isin(trdf.paragraph.tolist())]
# print(tedf.target.value_counts())

In [ ]:
# train1, train0  = art1[:int(0.65*len(art1))], art2[:int(0.8*len(art2))]
# print(len(train1), len(train0))
# train1.extend(train0)
# print(len(train1))
# test1, test0 = art1[int(0.65*len(art1)):], art2[int(0.8*len(art2)):] 
# print(len(test1), len(test0))
# test1.extend(test0)
# print(len(test1))# len(test_articles))

# any(i in train1 for i in test1)

In [ ]:
df_train , df_test = train_test_split(tdf, stratify=tdf['target'], random_state=69, test_size=0.25)
# df_train, df_test = tdf[tdf['article'].isin(train1)], tdf[tdf['article'].isin(test1)]
# df_train, df_test = trdf.copy(), tedf.copy()
df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)
print(df_train.shape, df_test.shape)
print(df_train.target.value_counts(), df_test.target.value_counts())
# del tdf

(13644, 3) (4548, 3)
1    9151
0    4493
Name: target, dtype: int64 1    3050
0    1498
Name: target, dtype: int64


In [ ]:
df_train.sample(frac=1).reset_index(drop=True, inplace=True)
df_test.sample(frac=1).reset_index(drop=True, inplace=True)
print(df_train.shape, df_test.shape)

(13644, 3) (4548, 3)


In [ ]:
!git lfs install

!git clone https://huggingface.co/pdelobelle/robbert-v2-dutch-base

# if you want to clone without large files – just their pointers
# prepend your git clone with the following env var:
GIT_LFS_SKIP_SMUDGE=1 


git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'robbert-v2-dutch-base'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 45 (delta 22), reused 0 (delta 0)
Unpacking objects: 100% (45/45), done.


In [ ]:
## Check if Cuda is Available
print(torch.cuda.is_available())
# specify GPU
device = torch.device("cuda")
print(torch.cuda.get_device_name(0))

True
Tesla T4


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base a

In [ ]:
class RobBERTDataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.data = dataframe
    self.article = self.data.article.tolist()
    self.text = self.data.paragraph
    self.targets = self.data.target
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.text)

  def __getitem__(self, idx):
    text = str(self.text[idx])
    text = " ".join(text.split())

    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        padding="max_length",
        truncation = True,
        max_length = self.max_len,
        return_token_type_ids=False
    )

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    # token_type_ids = inputs['token_type_ids']

    return {
        # 'article' : self.article,
        'input_ids': torch.tensor(ids, dtype=torch.long),
        'attention_mask': torch.tensor(mask, dtype=torch.long),
        # 'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long, device=device),
        'label': torch.tensor(self.targets[idx], dtype=torch.long)
    }

In [ ]:
max_len=512
train_dataset = RobBERTDataset(df_train, tokenizer, max_len)
test_dataset = RobBERTDataset(df_test, tokenizer, max_len)

In [ ]:
class RobBERT_Arch(nn.Module):
    def __init__(self, robbert):
      super(RobBERT_Arch, self).__init__()
      self.robbert = robbert
      # self.dropout = nn.Dropout(0.1)
      # self.relu =  nn.ReLU()
      # self.fc1 = nn.Linear(768,2)#change the 2 to 384 when using fc2 in forward function
      # self.fc2 = nn.Linear(384,2)
      # self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, sent_id, mask): #all commented when using modelforsequenceclassification version
      out = self.robbert(sent_id, mask)
      # print(out)
      # x = self.fc1(out.pooler_output)
      # x = self.dropout(x)
      # x = self.fc2(x)
      # x = self.softmax(out)
      # y = out.pooler_output      
      return out

In [ ]:
# Defining some key variables that will be used later on in the training
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 12

In [ ]:
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=TRAIN_BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=VALID_BATCH_SIZE)

In [ ]:

#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(df_train.target.tolist()), df_train.target.tolist())
# class_wts = [1.25, 1] 
print(class_wts)
# convert class weights to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

[1.5183619 0.7454923]


In [ ]:
# pass the pre-trained BERT to our define architecture
roberta_model = RobBERT_Arch(model)

# push the model to GPU
roberta_model = roberta_model.to(device)
# define the optimizer
param_optimizer = list(roberta_model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]


In [ ]:
def loss_fn(outputs, targets, weights):
    # return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))    
    cross_entropy = nn.CrossEntropyLoss(weights)
    return cross_entropy(outputs, targets)

In [ ]:
def train_loop_fn(dataloader, model, optimizer, device, weights, scheduler, scaler, iters_to_accumulate):
    model.train()
    total_loss, total_accuracy = 0, 0
    # empty list to save model predictions
    total_preds=[]
 
    for step, batch in tqdm(enumerate(dataloader), total=len(dataloader), leave=True):
        # push the batch to gpu
        batch = [batch[str(r)].to(device) for r in batch]
        input_id, mask, targets = batch
        #forward pass
        with amp.autocast():
            output = model(input_id, mask)
            output = output.logits
            # print(output, targets)
            loss = loss_fn(output, targets, weights)
            loss = loss/iters_to_accumulate

        total_loss += loss.item()
        scaler.scale(loss).backward()

        if (step + 1) % iters_to_accumulate == 0:
            # If you attempted to clip without unscaling, the gradients’ norm/maximum magnitude 
            # would also be scaled, so your requested threshold (which was meant to be the threshold for unscaled gradients) would be invalid.
            scaler.unscale_(optimizer)
            # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            optimizer.zero_grad()         
        # model predictions are stored on GPU. So, push it to CPU
        preds = output.detach().cpu().numpy()
        # append the model predictions
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

In [ ]:
def eval_fn(eval_dataloader, model, device):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, total=len(eval_dataloader), leave=True):
            # push the batch to gpu
        
            batch = [batch[str(r)].to(device) for r in batch]
            input_id, mask, targets = batch

            outputs = model(input_id, mask)
            outputs = outputs.logits
            
            true_labels.extend(targets.cpu().detach().numpy().tolist())
            preds.extend(outputs.cpu().detach().numpy().tolist())

    # reshape the predictions in form of (number of samples, no. of classes)
    # preds  = np.concatenate(preds, axis=0)

    return preds, true_labels

In [ ]:
torch.cuda.empty_cache()
# torch.load('robbert_ep3.pth')
# roberta_model.load_state_dict(torch.load('robbert_ep5.pth', map_location=device))

In [ ]:
scaler = amp.GradScaler()
EPOCHS = 5

iters_to_accumulate = 25
num_train_steps = int(len(train_dataset) / (TRAIN_BATCH_SIZE * iters_to_accumulate)  * EPOCHS)
print(num_train_steps)

optimizer = AdamW(optimizer_parameters, lr=1e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=70, num_training_steps=num_train_steps
)
# best_score = 0
for epoch in range(EPOCHS):
    _, tr_preds = train_loop_fn(train_dataloader,roberta_model, optimizer, device, weights, scheduler, scaler, iters_to_accumulate)
    # test_preds, labels = eval_fn(test_dataloader, roberta_model, device)
    # preds = np.argmax(test_preds, axis = 1)
    # F1_score = f1_score(labels, preds)
    # if F1_score > best_score:
    #     torch.save(roberta_model.state_dict(), 'robbert_ep' + str(EPOCHS) + '.pth')
    #     best_accuracy = accuracy

  0%|          | 0/1137 [00:00<?, ?it/s]

227


100%|██████████| 1137/1137 [09:38<00:00,  1.96it/s]


In [ ]:
!nvidia-smi

Tue Feb 23 01:39:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    43W /  70W |  10850MiB / 15109MiB |     56%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

test_preds, labels = eval_fn(test_dataloader, roberta_model, device)
print(test_preds[-3:])


100%|██████████| 379/379 [02:45<00:00,  2.30it/s]

[[-2.9740633964538574, 2.7524123191833496], [-1.780281662940979, 1.6981703042984009], [-2.7634119987487793, 2.6323885917663574]]


In [ ]:
preds = np.argmax(test_preds, axis = 1)
print(preds)
print('F1-score: ', f1_score(labels, preds))
print(classification_report(labels, preds))
print(accuracy_score(labels, preds))

[0 1 1 ... 1 1 1]
F1-score:  0.9354341496831651
              precision    recall  f1-score   support

           0       0.82      0.96      0.88      1498
           1       0.98      0.90      0.94      3050

    accuracy                           0.92      4548
   macro avg       0.90      0.93      0.91      4548
weighted avg       0.93      0.92      0.92      4548

0.9171064204045735


In [ ]:
# model_preds = tr_preds.append(test_preds)
# print(test_preds)
res = pd.DataFrame()
res['preds'] = preds
res['true_label'] = labels
print(res.shape)
# print(res.iloc[0:40, :]) 

(4548, 2)


In [ ]:
print(res.true_label.value_counts())
print(res.preds.value_counts())

1    3050
0    1498
Name: true_label, dtype: int64
1    2789
0    1759
Name: preds, dtype: int64


In [ ]:
# torch.save(roberta_model.state_dict(), 'robbert_ep5.pth')

In [ ]:
df_test['preds'] = preds
print(accuracy_score(df_test['target'], df_test['preds']))

0.9171064204045735


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
print(df_test.iloc[:5, 1:])
df_res = df_test.groupby('article')['target', 'preds'].transform(max).reset_index()
print(df_res.shape)

                                           paragraph  target  preds
0  § 2. De televisie-omroepinstellingen die onder...       0      0
1  - hebben de rechters-verslaggevers F. Daoût en...       1      1
2  40 van de frequenties op complementaire wijze ...       1      1
3                    - is de zaak in beraad genomen.       1      1
4  112. Gelet op een en ander dient op de eerste ...       0      0
(4548, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [ ]:
df_res['error'] = abs(df_res.target - df_res.preds)
print(df_res.error.value_counts(normalize=True))
print(df_res.target.value_counts())
print(df_res.preds.value_counts())

0    0.989006
1    0.010994
Name: error, dtype: float64
1    4513
0      35
Name: target, dtype: int64
1    4479
0      69
Name: preds, dtype: int64


In [ ]:
2789+1759

4548